In [1]:
import pandas as pd

# 1. Data loading

In [2]:
SHEET_NAMES = [
    "Trivia",
    "Smarther Than a 5th Grader",
    "Question Needs Category Removed",
    # "Millionaire - Easy Q", 
    # "Millionaire - Hard Q"
    ]

In [3]:
datasets = {}
for sheet_name in SHEET_NAMES:
    datasets[sheet_name] = pd.read_excel("data/Trivia-Printable.xlsx", sheet_name=sheet_name)


In [4]:
concatenated_df = pd.DataFrame()
for sheet_name in SHEET_NAMES:
    if sheet_name == "Question Needs Category Removed":
        new_df= datasets[sheet_name]
        new_df["Sheet Name"] = sheet_name
        concatenated_df = pd.concat([concatenated_df, datasets[sheet_name]], axis=0, ignore_index=True)
        continue
    for suffix in ["", ".1", ".2"]:
        new_df = datasets[sheet_name][["Question" + suffix, "Answer" + suffix]]
        new_df.columns = ["Question", "Answer"]
        if "Category"+suffix in datasets[sheet_name].columns:
            new_df["Category"] = datasets[sheet_name]["Category"+suffix]
        new_df = new_df.copy()
        new_df.loc[:, 'Sheet Name'] = sheet_name
        concatenated_df = pd.concat([concatenated_df, new_df], axis=0, ignore_index=True)

/tmp/ipykernel_21710/2659253914.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Category"] = datasets[sheet_name]["Category"+suffix]
/tmp/ipykernel_21710/2659253914.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Category"] = datasets[sheet_name]["Category"+suffix]
/tmp/ipykernel_21710/2659253914.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [5]:
concatenated_df.Category = concatenated_df['Category'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [6]:
concatenated_df['Category'] = concatenated_df['Category'].replace(
    {"Mathematics": "Mathematics & Geometry", 
     "Science": "Science", "History": "History", "Geography": "Geography", "Art": "Art", "Music": "Music", "Literature": "Literature", "Other": "Other"}
)

# 2. LLM challenger Answer a question

In [7]:
from json import load
import torch
from transformers import pipeline
from src.llm import SELECTED_LLMS
from src.answer import answer_questions

question = "What is the capital of Sri Lanka?"

for llm in SELECTED_LLMS[2:]:
    print(llm.name)
    print(answer_questions([question], llm)[0])


/home/lambert-bruyas/miniconda3/envs/gpt_pursuit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


google/flan-t5-small
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/google/flan-t5-small


Device set to use cpu


[{'generated_text': 'san juan'}]
google/flan-t5-base
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/google/flan-t5-base


Device set to use cpu


[{'generated_text': 'sri lanka'}]
google/gemma-2-2b-it
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/google/gemma-2-2b-it


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  5.56it/s]
Device set to use cpu
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


[{'generated_text': [{'role': 'user', 'content': 'You are participating in a trivia contest. Answer the following question: What is the capital of Sri Lanka?'}, {'role': 'assistant', 'content': 'The capital of Sri Lanka is **Sri Jayawardenepura Kotte**. \n'}]}]
Qwen/Qwen2.5-1.5B-Instruct
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/Qwen/Qwen2.5-1.5B-Instruct


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.50it/s]
Device set to use cpu


[{'generated_text': [{'role': 'user', 'content': 'You are participating in a trivia contest. Answer the following question: What is the capital of Sri Lanka?'}, {'role': 'assistant', 'content': 'The capital of Sri Lanka is Colombo.'}]}]


In [48]:

loaded_pipe = pipeline("text-generation", model="./models/"+model_kwargs["model"])



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.94it/s]
Device set to use cpu


In [49]:

loaded_pipe([{"role":"user", "content":PROMPT.format(
    question="What is the capital of Sri Lanka?"
    )}])

[{'generated_text': [{'role': 'user',
    'content': 'You are participating in a trivia contest. Answer the following question: What is the capital of Sri Lanka?'},
   {'role': 'assistant', 'content': 'The capital of Sri Lanka is Colombo.'}]}]

In [51]:
loaded_pipe(PROMPT.format(
    question="What is the capital of Sri Lanka?"
    ))

[{'generated_text': 'You are participating in a trivia contest. Answer the following question: What is the capital of Sri Lanka? - Colombo - Jaffna - Batticaloa - Trincomalee\nThe answer'}]

In [37]:
generator_pipeline = pipeline(
    "text2text-generation",
    model='google/flan-t5-small',
    tokenizer='google/flan-t5-small', # Explicitly specifying tokenizer is good practice
)

Device set to use cpu


In [43]:
generator_pipeline("What is the capital of USA?")

[{'generated_text': 'san diego'}]

In [44]:
loaded_pipe("What is the capital of Sri Lanka?")

[{'generated_text': 'What is the capital of Sri Lanka?'}]

### 2.2 Answer questions

In [38]:
concatenated_df.loc[concatenated_df['Category'] == 'Mathematics & Geometry']

,Question,Answer,Category,Sheet Name,Difficulty
1334,The angles inside a square total _______ degrees.,360,Mathematics & Geometry,Trivia,NaN
2276,How many nickles are there in $2.25,45,Mathematics & Geometry,Trivia,NaN
3451,The first antiderivative of acceleration is:,Velocity,Mathematics & Geometry,Trivia,NaN
4035,An angle greater than 180 degrees and less tha...,Reflex,Mathematics & Geometry,Trivia,NaN
4620,What is the maximum number of integer degrees ...,359,Mathematics & Geometry,Trivia,NaN
5924,A triangle with three equal sides is called __...,Equilateral,Mathematics & Geometry,Trivia,NaN
6781,Two angles that total 180 degrees are called _...,Supplementary,Mathematics & Geometry,Trivia,NaN
6879,What is the minimum number of integer degrees ...,91,Mathematics & Geometry,Trivia,NaN
7224,Name the number system which uses only the sym...,The binary system,Mathematics & Geometry,Trivia,NaN
10585,If you cut through a solid sphere what shape w...,Circle,Mathematics & Geometry,Trivia,NaN


In [39]:
sample_questions = concatenated_df.loc[
    concatenated_df['Category'] == 'Mathematics & Geometry'].sample(5)

In [40]:
for i, row in sample_questions.iterrows():
    question_prompt = PROMPT.format(
        question=row["Question"]
    )
    text = loaded_pipe(question_prompt, max_length=100)[0]["generated_text"]
    sample_questions.loc[i, model_kwargs["model"]] = text.replace(question_prompt, "")

In [41]:
sample_questions

,Question,Answer,Category,Sheet Name,Difficulty,google/gemma-2-2b-it
1334,The angles inside a square total _______ degrees.,360,Mathematics & Geometry,Trivia,NaN,\n\nHere's the breakdown:\n\n* **Squares have ...
30532,An angle greater than 90 degrees is said to be...,Obtuse,Mathematics & Geometry,Trivia,NaN,\n\nHere are the possible answers:\n\na) Acute...
4035,An angle greater than 180 degrees and less tha...,Reflex,Mathematics & Geometry,Trivia,NaN,\n\n**Answer:** **Supplementary** angle \n
13966,"If you count from 1 to 100, how many 7's will ...",20,Mathematics & Geometry,Trivia,NaN,\n\n**Answer:** 16 \n
27136,Who proved Fermat's Last Theorem ?,Andrew Wiles,Mathematics & Geometry,Trivia,NaN,\n\n**Answer:** Andrew Wiles \n


In [42]:
# Set pandas display options to show all text in columns
pd.set_option('display.max_colwidth', None)  # Show full text in each cell
pd.set_option('display.width', None)         # Auto-detect display width
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines

# Display the sample questions again with the new display settings
sample_questions


,Question,Answer,Category,Sheet Name,Difficulty,google/gemma-2-2b-it
1334,The angles inside a square total _______ degrees.,360,Mathematics & Geometry,Trivia,NaN,\n\nHere's the breakdown:\n\n* **Squares have four equal sides and four right angles.**\n* **The sum of the interior angles of a triangle is 180 degrees.**\n* **The sum of the interior angles of a quadrilateral (four-sided shape) is 360 degrees.**\n\n**Answer:** 360 degrees \n
30532,An angle greater than 90 degrees is said to be _________.,Obtuse,Mathematics & Geometry,Trivia,NaN,\n\nHere are the possible answers:\n\na) Acute\nb) Obtuse\nc) Right\nd) Straight\n\n**Correct Answer:** b) Obtuse\n\n**Explanation:**\n\n* **Acute angles** are less than 90 degrees.\n* **Right angles** are exactly 90 degrees.\n* **Straight angles**
4035,An angle greater than 180 degrees and less than 360 degrees is a(n) ________ angle.,Reflex,Mathematics & Geometry,Trivia,NaN,\n\n**Answer:** **Supplementary** angle \n
13966,"If you count from 1 to 100, how many 7's will you come across?",20,Mathematics & Geometry,Trivia,NaN,\n\n**Answer:** 16 \n
27136,Who proved Fermat's Last Theorem ?,Andrew Wiles,Mathematics & Geometry,Trivia,NaN,\n\n**Answer:** Andrew Wiles \n
